In [ ]:
# 1. Use best short-term minimal model (for time prediction? prefer models with less parameters if v close scores) and train it with complete dataset
# 2. Increase parameters for all rows and use trained model to predict extreme snowmelt
# 3. Separate analyses including only sites where variables are within the values of the training dataset and with all data
# 4. also need some robustness checks - look at different flow model outputs and see how consistent they are